In [1]:
import os

import minicons.cwe
!pip install minicons

  Using cached pandas-2.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached urllib3-1.26.20-py2.py3-none-any.whl.metadata (50 kB)
Using cached pandas-2.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached urllib3-1.26.20-py2.py3-none-any.whl (144 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.2
    Not uninstalling urllib3 at /nix/store/dcgjkj2z9zlmis91fq9py6c49rvasbnb-python3.12-urllib3-2.2.2/lib/python3.12/site-packages, outside environment /home/leo/git/miniconsPlayground/venv
    Can't uninstall 'urllib3'. No files were found to uninstall.
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Not uninstalling pandas at /nix/store/6if4ab4fy227ww2ay90ddlnyp4d8dzgh-python3.12-pandas-2.2.2/lib/python3.12/site-packages, outside environment /home/leo/git/miniconsPlayground/venv
    Can't uninstall 'pandas'. No files were found to uninstall.

[notice] A 

In [2]:
from minicons import cwe


In [3]:
model = cwe.CWE('bert-base-uncased')


context_words = [("I went to the bank to withdraw money.", "bank"), 
                 ("i was at the bank of the river ganga!", "bank")]

print(model.extract_representation(context_words, layer = 12))

''' 
tensor([[ 0.5399, -0.2461, -0.0968,  ..., -0.4670, -0.5312, -0.0549],
        [-0.8258, -0.4308,  0.2744,  ..., -0.5987, -0.6984,  0.2087]],
       grad_fn=<MeanBackward1>)
'''

/home/leo/git/miniconsPlayground/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tensor([[ 0.5737, -0.5295, -0.0294,  ..., -0.7193, -0.3905, -0.1534],
        [-0.8258, -0.4308,  0.2744,  ..., -0.5987, -0.6984,  0.2087]])


' \ntensor([[ 0.5399, -0.2461, -0.0968,  ..., -0.4670, -0.5312, -0.0549],\n        [-0.8258, -0.4308,  0.2744,  ..., -0.5987, -0.6984,  0.2087]],\n       grad_fn=<MeanBackward1>)\n'

In [4]:
# if model is seq2seq:
model = cwe.EncDecCWE('t5-small')

print(model.extract_representation(context_words))

'''(last layer, by default)
tensor([[-0.0895,  0.0758,  0.0753,  ...,  0.0130, -0.1093, -0.2354],
        [-0.0695,  0.1142,  0.0803,  ...,  0.0807, -0.1139, -0.2888]])
'''

tensor([[-0.2598, -0.1003,  0.0277,  ...,  0.0383, -0.2195, -0.2411],
        [-0.3215, -0.1050, -0.3203,  ..., -0.2163, -0.1055, -0.1272]])


'(last layer, by default)\ntensor([[-0.0895,  0.0758,  0.0753,  ...,  0.0130, -0.1093, -0.2354],\n        [-0.0695,  0.1142,  0.0803,  ...,  0.0807, -0.1139, -0.2888]])\n'

# Load stimuli

In [3]:
import os

stimuli_folder = "./stimuli"
stimuli = []
for file in os.listdir(stimuli_folder):
    with open(stimuli_folder + "/" + file, 'r') as stimulus:
        text = stimulus.read().replace('\n', '')
        stimuli.append(text)

story_09.txt:
So a lot of people are raised with a religion, and I'm one of those people that was raised with two. {BR} There was our official religion, my family is and I was at the time conservative evangelical christians. {BR} But then there was our unofficial faith and hope we had in Mary Kay cosmetics. {LG} Um {LS} many of the women in my family had been Mary Kay beauty consultants since I was {BR} a child and {BR} I loved Jesus as long as I could remember but that Mary Kay thing was {BR} I was hoping to dodge that bullet. Um I was this brainy grunge girl from Boulder county, I wore {BR} hemp jewelry and clothes from the Salvation army, {BR} I had moral obliga um objections to wearing make up. {BR} And when I got a scholarship to go out of state for college, I thought I'd made my big escape. {BR} But my last visit home, I listened to one recruiting tape too many {LG}, and the logic and reason finally got to me and I signed the dotted line. And I started my business as a Mary Kay b